In [2]:
import cv2
import numpy as np
import math

def GetEyes(frame):
    # Umwandeln in Graustufenbild
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(grayFrame, scaleFactor=1.1, minNeighbors=4)
    print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords der Augen: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("delta X Augen:", deltaX, "delta Y Augen: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand der Augen: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, angle, spacing):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-spacing/60)*100 #scale des Bildes in % CutFace hat einen Abstand von 60
    print("Skalierung in Prozent", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result

#Überlappen der Beiden Bilder, damit die Augen aufeinander liegen
def Overlap(image1, image2):
    y = 480
    x = 640

    print("X Finale Image: ", x, " Y Finale Image: ", y)
    #Neues np Array anlegen, dass die größte Breite und die Größte Höhe hat
    result = np.zeros((y, x, 3), dtype=np.uint8)

    #Offset mit dem das GesichtFrau Bild verschoben werden soll
    xOffset = 100
    yOffset = 100

    #Durchiterrieren aller Pixel um das Finale Bild generieren zu lassen result ist ein Bild mit RGB Kanal
    for i in range(x):
        for j in range(y):
            #try fängt indizierungen auserhalb des erlaubten Bereichs ab
            try:
                if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                    result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

                else:
                    result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
            except:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
    return result
  
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()

    #Augen Position erfassen
    eyes = GetEyes(frame)

    for (x, y, w, h) in eyes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    #Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
    eyesPos = list()
    for i in range(len(eyes)):
        eyesPos.append((eyes[i][0],eyes[i][1]))

    #Berechnen des Winkels und Abstand der Augen
    if len(eyesPos) == 2:
        angle, spacing = CalcAngleAndSpacing(eyesPos)
        print("Winkel", angle)
        print("Abstand der Augen", spacing)
        face = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)
        #Gesicht Rotieren und auf den Augenabstad skalieren
        rotatet = RotateAndScale(face, angle, spacing)
        #finalFrame = Overlap(frame, rotatet)
    
    #Anzeigen des Frames
    #print("Frame Shape: ", frame.shape)
    #cv2.imshow('Pic', rotatet)
    cv2.imshow('Video', frame)
    #cv2.imshow('Video', finalFrame)
      
    #Beenden mit der Taste q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

Augen:  [[361 126  44  44]
 [286 118  57  57]
 [326 309  47  47]]
Augen:  [[361 125  44  44]
 [285 117  60  60]]
Cords der Augen:  [(361, 125), (285, 117)]
delta X Augen: 76 delta Y Augen:  8
Abstand der Augen:  76.4198926981712
Winkel 6.053509518097258
Abstand der Augen 76.4198926981712
Skalierung in Prozent 127.36648783028532


QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread (0x275b3d0).
Cannot move to target thread (0x2a354f0)

QObject::moveToThread: Current thread (0x2a354f0) is not the object's thread

Augen:  [[361 126  44  44]
 [286 117  58  58]]
Cords der Augen:  [(361, 126), (286, 117)]
delta X Augen: 75 delta Y Augen:  9
Abstand der Augen:  75.53806987208503
Winkel 6.908687118692551
Abstand der Augen 75.53806987208503
Skalierung in Prozent 125.89678312014172
Augen:  [[362 127  42  42]
 [287 118  57  57]]
Cords der Augen:  [(362, 127), (287, 118)]
delta X Augen: 75 delta Y Augen:  9
Abstand der Augen:  75.53806987208503
Winkel 6.908687118692551
Abstand der Augen 75.53806987208503
Skalierung in Prozent 125.89678312014172
Augen:  [[360 126  44  44]
 [286 117  58  58]]
Cords der Augen:  [(360, 126), (286, 117)]
delta X Augen: 74 delta Y Augen:  9
Abstand der Augen:  74.54528824815154
Winkel 7.002968595105138
Abstand der Augen 74.54528824815154
Skalierung in Prozent 124.24214708025256
Augen:  [[361 126  43  43]
 [284 114  66  66]]
Cords der Augen:  [(361, 126), (284, 114)]
delta X Augen: 77 delta Y Augen:  12
Abstand der Augen:  77.92945527847606
Winkel 9.002210770505524
Abstand der 

In [27]:
deltaXMin = 0
finalEyes = list()
for i in range(2,len(eyesPos)+1):
    print(i)
    delta = eyesPos[i-1][0] - eyesPos[i][0]
    if deltaXMin == 0 or delta < deltaXMin:
        deltaXMin = delta
        finalEyes = []
        finalEyes.append(eyesPos[i-1])
        finalEyes.append(eyesPos[i])

2
3
4
5
